In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import re


pd.set_option('display.max_columns', None)

In [36]:
# df = pd.read_csv('./dataset/cars_train.csv', sep='\t', encoding='utf_16')

In [53]:
# Ler dados de treino
df = pd.read_csv('./dataset/cars_train.csv', sep='\t', encoding='utf_16')
# df = df_ft_eng.copy()
# Transformar para inteiro
df['ano_modelo'] = df.ano_modelo.astype(int)
df['num_fotos'] = df.num_fotos.fillna(0).astype(int)

# Transformar em booleanos
df['dono_aceita_troca'] = df.dono_aceita_troca.notnull().astype(int)
df['veiculo_único_dono'] = df.veiculo_único_dono.notnull().astype(int)
df['revisoes_concessionaria'] = df.revisoes_concessionaria.notnull().astype(int)
df['ipva_pago'] = df.ipva_pago.notnull().astype(int)
df['veiculo_licenciado'] = df.veiculo_licenciado.notnull().astype(int)
df['garantia_de_fábrica'] = df.garantia_de_fábrica.notnull().astype(int)
df['revisoes_dentro_agenda'] = df.revisoes_dentro_agenda.notnull().astype(int)
df['entrega_delivery'] = df.entrega_delivery.astype(int)
df['troca'] = df.troca.astype(int)
df['blindado'] = df.blindado.map({'N': 0, 'S': 1})

# Colunas descartadas
df.drop(columns=['veiculo_alienado'], inplace=True)
df.drop(columns=['elegivel_revisao'], inplace=True)

# Remapeando tipo_vendedor e alterando o nome da variável
df['tipo_vendedor'] = df.tipo_vendedor.map({'PJ': 1, 'PF': 0})
df.rename(columns={'tipo_vendedor':'vendedor_PJ'}, inplace=True)

In [38]:
# Copiando data frame original
df_ft_eng = df.copy()
# os 3 primeiros digitos referem-se as cilindradas do motor
# para os carros elétricos, que não tem essa identificação, o motor foi considerado como igual a 0
df_ft_eng['motor'] = df_ft_eng.versao.str.slice(0, 3).apply(lambda x: float(x) if re.findall('^\d\.\d$', x) else 0.0)

combustivel = df_ft_eng.versao.apply(lambda car: 'gasolina' if re.findall('GAS', car) else\
                                    ('flex' if re.findall('FLEX|TETRAFUEL', car) else \
                                    ('diesel' if re.findall('DIESEL', car) else \
                                    ('eletrico' if re.findall('ELETRI|ELÉTRI|ELECTRI|RECHARG', car) else \
                                    ('hibrido' if re.findall('HÍBRID|HYBRID|HIBRID', car) else \
                                    ('flex' if re.findall('1\.', car) else 'gasolina')))))) # carros da volkswagen que são flex

df_ft_eng['combustivel'] = combustivel
df_ft_eng['4x4'] = df_ft_eng.versao.str.contains('4X4')
df_ft_eng['turbo'] = df_ft_eng.versao.str.contains('TURBO')

In [41]:
df_final = df_ft_eng.copy()
df_final['tipo'] = df_final['tipo'].astype('category').cat.codes
df_final['cambio'] = df_final['cambio'].astype('category').cat.codes
df_final['cor'] = df_final['cor'].astype('category').cat.codes
df_final['anunciante'] = df_final['anunciante'].astype('category').cat.codes
df_final['combustivel'] = df_final['combustivel'].astype('category').cat.codes

df_final['4x4'] = df_final['4x4'].astype(int)
df_final['turbo'] = df_final['turbo'].astype(int)

df_final['marca'] = df_final['marca'].astype('category').cat.codes
df_final['modelo'] = df_final['modelo'].astype('category').cat.codes
df_final['versao'] = df_final['versao'].astype('category').cat.codes
df_final['estado_vendedor'] = df_final['estado_vendedor'].astype('category').cat.codes
df_final.drop(columns='cidade_vendedor', inplace=True)
df_final['diferenca_ano'] = df_final.ano_modelo - df_final.ano_de_fabricacao
df_final.drop(columns=['marca', 'modelo', 'vendedor_PJ'])

X = df_final.drop(columns=['preco', 'id'])
y = df_final.preco

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=31)


In [42]:
df_final

,id,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,vendedor_PJ,estado_vendedor,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,preco,motor,combustivel,4x4,turbo,diferenca_ano
0,300716223898539419613863097469899222392,8,27,257,424,2017,2017,67772.0,4,4,5,0,0,0,23,3,0,0,0,0,1,1,1,0,0,74732.590084,1.6,2,0,0,0
1,279639842134129588306469566150288644214,8,18,133,876,2017,2017,62979.0,2,4,5,0,0,0,10,3,0,0,1,0,0,1,0,0,0,81965.332634,2.0,2,0,0,0
2,56414460810621048900295678236538171981,16,19,378,1402,2018,2019,44070.0,2,4,5,0,4,1,23,2,1,0,1,0,0,0,0,0,0,162824.814472,2.4,3,0,0,1
3,56862509826849933428086372390159405545,14,38,63,1087,2013,2015,85357.0,2,4,4,0,0,1,23,2,1,1,1,0,0,1,1,0,0,123681.358857,2.0,0,1,1,2
4,338980975753200343894519909855598027197,8,33,259,1076,2013,2015,71491.0,2,4,6,0,4,0,18,3,0,0,0,0,1,0,0,1,1,82419.763891,2.0,0,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29579,40985083650499509711752444068313401704,8,36,238,1529,2021,2021,8150.0,2,4,4,0,0,0,6,3,0,0,1,1,0,1,1,1,0,403015.289616,2.8,0,1,1,0
29580,164663177589009976281210123914772747797,8,38,319,22,2020,2020,17987.0,2,4,4,0,0,0,11,3,0,0,0,1,1,1,1,1,0,88978.080497,1.0,2,0,0,0
29581,170866287186047465472146212778882034162,8,36,238,1553,2019,2019,44742.0,2,4,4,0,1,0,3,3,0,0,1,0,0,0,0,0,0,218807.648664,2.8,0,1,0,0
29582,208909739337040262669325459666578363437,8,28,8,394,2021,2022,35376.0,2,4,5,0,0,1,23,2,1,0,1,1,0,1,1,0,0,68495.990693,1.6,2,0,0,1


In [43]:
rfr = RandomForestRegressor(**{'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10})

In [44]:
rfr.fit(X_train, y_train)

RandomForestRegressor(min_samples_leaf=2, min_samples_split=10)

In [45]:
rfr.score(X_val, y_val)
# 0.7379896649768927 - todas
# 0.7376579977714588 - sem marca e modelo

0.7395226937141955

In [46]:
pd.Series(rfr.feature_importances_, index=X_train.columns).sort_values()

vendedor_PJ                0.000897
garantia_de_fábrica        0.001360
entrega_delivery           0.001398
troca                      0.001568
anunciante                 0.001762
4x4                        0.001826
blindado                   0.002546
ipva_pago                  0.002975
veiculo_licenciado         0.003058
veiculo_único_dono         0.003198
revisoes_dentro_agenda     0.003285
revisoes_concessionaria    0.003538
diferenca_ano              0.004800
num_portas                 0.005468
dono_aceita_troca          0.005905
num_fotos                  0.010510
estado_vendedor            0.012204
turbo                      0.013764
motor                      0.016402
cor                        0.017091
cambio                     0.018682
marca                      0.030866
ano_modelo                 0.045315
modelo                     0.045328
ano_de_fabricacao          0.048815
combustivel                0.059344
tipo                       0.077312
hodometro                  0

In [7]:
# Ler dados de teste
df_test_raw = pd.read_csv('./dataset/cars_test.csv', sep='\t', encoding='utf_16')

df_test = df_test_raw.copy()

# Transformar para inteiro
df_test['ano_modelo'] = df_test.ano_modelo.astype(int)
df_test['num_fotos'] = df_test.num_fotos.fillna(0).astype(int)

# Transformar em booleanos
df_test['dono_aceita_troca'] = df_test.dono_aceita_troca.notnull().astype(int)
df_test['veiculo_único_dono'] = df_test.veiculo_único_dono.notnull().astype(int)
df_test['revisoes_concessionaria'] = df_test.revisoes_concessionaria.notnull().astype(int)
df_test['ipva_pago'] = df_test.ipva_pago.notnull().astype(int)
df_test['veiculo_licenciado'] = df_test.veiculo_licenciado.notnull().astype(int)
df_test['garantia_de_fábrica'] = df_test.garantia_de_fábrica.notnull().astype(int)
df_test['revisoes_dentro_agenda'] = df_test.revisoes_dentro_agenda.notnull().astype(int)
df_test['entrega_delivery'] = df_test.entrega_delivery.astype(int)
df_test['troca'] = df_test.troca.astype(int)
df_test['blindado'] = df_test.blindado.map({'N': 0, 'S': 1})

# Colunas descartadas
df_test.drop(columns=['veiculo_alienado'], inplace=True)
df_test.drop(columns=['elegivel_revisao'], inplace=True)

# Remapeando tipo_vendedor e alterando o nome da variável
df_test['tipo_vendedor'] = df_test.tipo_vendedor.map({'PJ': 1, 'PF': 0})
df_test.rename(columns={'tipo_vendedor':'vendedor_PJ'}, inplace=True)

df_test['tipo'] = df_test['tipo'].astype('category').cat.codes
df_test['cambio'] = df_test['cambio'].astype('category').cat.codes
df_test['cor'] = df_test['cor'].astype('category').cat.codes
df_test['anunciante'] = df_test['anunciante'].astype('category').cat.codes

df_test['marca'] = df_test['marca'].astype('category').cat.codes
df_test['modelo'] = df_test['modelo'].astype('category').cat.codes
df_test['versao'] = df_test['versao'].astype('category').cat.codes
df_test['estado_vendedor'] = df_test['estado_vendedor'].astype('category').cat.codes
df_test.drop(columns='cidade_vendedor', inplace=True)

X = df_test.drop(columns=['id'])

In [47]:
y_pred = rfr.predict(X_val)

In [48]:
carros_train = X_train.versao.sort_values().unique()
carros_val = X_val.versao

precos_carros = []

for carro, preco_real, preco_pred in zip(carros_val, y_val, y_pred):
    if carro not in carros_train:
        precos_carros.append([carro, preco_real, preco_pred])

precos_carros = pd.DataFrame(precos_carros, columns=['carro', 'preco_real', 'preco_pred'])
precos_carros

,carro,preco_real,preco_pred
0,1382,168632.668599,154259.762669
1,802,57145.641448,41043.467971
2,1288,85915.967032,89414.163192
3,985,185658.072314,228230.582005
4,1690,38715.747105,85808.218541
...,...,...,...
203,1382,160490.687523,151173.087306
204,706,57842.345509,55486.214560
205,1843,62805.845608,169174.717482
206,797,34159.328083,38077.583081


In [49]:
r2_score(precos_carros.preco_real, precos_carros.preco_pred)

0.5098265623983582

Teste com NLP

In [96]:
df2 = df.copy()
df2['tipo'] = df2['tipo'].astype('category').cat.codes
df2['cambio'] = df2['cambio'].astype('category').cat.codes
df2['cor'] = df2['cor'].astype('category').cat.codes
df2['anunciante'] = df2['anunciante'].astype('category').cat.codes

df2['marca'] = df2['marca'].astype('category').cat.codes
df2['modelo'] = df2['modelo'].astype('category').cat.codes
# df2['versao'] = df2['versao'].astype('category').cat.codes
df2['estado_vendedor'] = df2['estado_vendedor'].astype('category').cat.codes
df2.drop(columns='cidade_vendedor', inplace=True)

In [97]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
descriptions = vectorizer.fit_transform(df2.versao)
descriptions = pd.DataFrame(descriptions.toarray(), columns=vectorizer.get_feature_names_out())
df2 = pd.concat([df2.drop(columns=['id']), descriptions], axis=1)
df2['motor'] = df2.versao.str.slice(0, 3).apply(lambda x: float(x) if re.findall('^\d\.\d$', x) else 0.0)
# df2.drop(columns='versao', inplace=True)
df2['versao'] = df2['versao'].astype('category').cat.codes
df2

,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,tipo,blindado,cor,vendedor_PJ,estado_vendedor,anunciante,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,preco,10,100,104,10at,110,120,120cv,122cv,12v,130cv,131cv,140cv,142cv,143cv,150cv,155cv,16,164,16l,16v,170,170cv,180cv,182cv,183cv,184cv,18i,18l,18v,1a11,200,200cv,202cv,20i,20v,21,211cv,24v,25,250,2500,263,270,270cv,285cv,28i,2p,2ss,2wd,300,30d,30s13,30v,311,313,314,32v,330,340cv,35,350,35i,36v,3p,40,40v,415,416,45,4all,4d,4matic,4motion,4p,4s,4sport,4style,4wd,4x2,4x4,4you,50i,515,516,55,5m,60,6a,6i,6v,75,7g,7l,800kg,8v,90,94cv,97cv,9g,absolute,act,action,activ,activ7,active,activeflex,advance,advantage,adventure,air,all4,allgrip,allspace,allure,altis,alto,ambiente,ambition,amg,anos,at6,at8,at9,atacama,attack,attitude,attraction,attractive,audace,authentique,autobiography,automatico,automatizado,automático,automátizado,avant,avantgarde,awc,awd,b12p,base,be,bev,bi,biturbo,black,blackjack,bluehdi,bluemotion,bluetec,boost,bose,boxster,brabus,business,bva,cab,cabine,cabrio,cabriolet,cargo,cargo16v,carrera,cat,cayman,caçamba,cc,cd,cdi,ce,cgi,challenge,chassi,chassis,chic,chilli,cilindros,city,class,classic,clubman,colina,com,combinato,comfort,comfortline,compact,competition,confortline,connect,conquest,conversível,cooper,copa,country,coupe,coupé,coupê,cross,crosstrek,crossway,crx,cs,cult,curl,curto,custom,cvt,d180,d240,d300,d5,dark,dci,dct,de,del,design,diamond,diesel,direct,dlx,do,drive,dsg,dx,dynamic,dynamique,eagle,easy,eat6,ecoboost,econo,economy,ecoshift,ecotec,edition,edição,edrive,efi,el,electric,electronic,elegance,elegant,eletronic,elevado,elite,elx,elétrico,endurance,enduro,engineered,eq,escapade,especial,essence,estate,ev,ev400,evo,evolution,ex,ex2,ex3,excellence,exclusive,executive,exl,express,expression,exr,exs,extra,extreme,f1,fastback,feel,feline,fifa,fifty,fire,firefly,first,flex,flexone,flexstart,forest,freedom,freeride,freestyle,fsi,full,furgão,fwd,gas,gasolina,gdi,gear,geartronic,ghia,gl,gli,gls,glx,gp,gr,gran,grand,graphite,griffe,gsr,gsx,gt,gtdi,gte,gti,gts,guard,gx,h4,h6,hard,hatch,hd,hdi,hemi,hgt,hi,high,highline,highway,hl,hle,hls,hpe,hpi,hr,hse,hybrid,híbrido,i4,i6,iconic,ie,iflex,ii,iii,impetus,impress,in,ingenium,inscription,intense,intensive,intercoler,intercooler,iv,jet,jetflex,john,joy,k788,kinetic,kompressor,koup,kw,l1h1,l2h1,l2h2,l3h2,label,landmark,laramie,laredo,last,launch,launching,le,life,like,limitada,limited,line,lineartronic,live,locker,longitude,longo,look,lotação,ls,lt,ltz,lugares,luxe,luxury,lx,lxl,lxr,lxs,m40i,m50d,m50i,mach,manual,maxicargo,maxx,mct,mhev,mi,midnight,mille,million,minibus,mivec,moab,momentum,motion,move,mpfi,mpi,msi,multiair,multidrive,multijet,multitronic,mundo,médio,night,ocean,open,origine,outdoor,outsider,overland,p250,p300,p380,p550,p8,pack,pallas,passion,pdk,pepper,performance,personal,phev,picasso,pick,platinum,plus,polestar,portfolio,power,powershift,precision,premier,premium,presence,prestige,prime,pro,progressive,pulse,pure,q4,quattro,racing,ralliart,rallye,ranch,rbw,rebel,recharge,rex,rigido,rio,rip,road,roadster,robust,rocam,rock,rs,rt,rts,run,rx,rxs,rxt,rígido,sahara,sce,sd4,sda,sdrive18i,sdrive20i,sdrive30i,sdv8,se,security,sedan,sel,selectshift,sem,sense,series,sfi,shadow,shift,shine,si,si4,sidi,sierra,sigma,sl,slt,smart,so,sol,spe4,spec,special,speed,speedshift,spider,spirit,sport,sport6,sportback,sporting,sportline,sportrac,sr,srv,srx,ss,start,std,steptronic,stepway,sti,storm,street,stx,style,super,supercharged,surf,sv,svautobiography,svr,sw,sx,sx3,sxt,série,t270,t3,t4,t5,t6,t8,ta,take,targa,tci,td,td350,td380,td4,td6,tdi,tdv6,tdv8,tech,tecno,tendance,teto,tetrafuel,tfsi,tgdi,tgv,thp,ti,tiptronic,titanium,top,tornado,torq,total,totalflex,touring,track,trailhawk,trekking,trend,trendline,tronic,trooper,tropical,tropivan,tsi,turbo,turbocharged,turismo,tweed,twi

In [150]:
X = df2.drop(columns='preco')
y = df2.preco

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.9, random_state=31)

In [151]:
rfr.fit(X_train, y_train)
rfr.score(X_val, y_val)

0.6757971037942072

In [152]:
features_importance = pd.Series(data=rfr.feature_importances_, index=rfr.feature_names_in_)

In [153]:
features_importance.sort_values(ascending=False)

versao               0.281843
hodometro            0.208207
flex                 0.076327
tipo                 0.072437
ano_de_fabricacao    0.043765
                       ...   
ii                   0.000000
140cv                0.000000
i4                   0.000000
142cv                0.000000
hemi                 0.000000
Length: 701, dtype: float64

In [212]:
df1 = df.copy()
df1['tipo'] = df1['tipo'].astype('category').cat.codes
df1['cambio'] = df1['cambio'].astype('category').cat.codes
df1['cor'] = df1['cor'].astype('category').cat.codes
df1['anunciante'] = df1['anunciante'].astype('category').cat.codes

df1['marca'] = df1['marca'].astype('category').cat.codes
df1['modelo'] = df1['modelo'].astype('category').cat.codes
df1['versao'] = df1['versao'].astype('category').cat.codes
df1['estado_vendedor'] = df1['estado_vendedor'].astype('category').cat.codes
df1.drop(columns='cidade_vendedor', inplace=True)

X = df1.drop(columns='preco')
y = df1.preco

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=31)

In [213]:
rfr.fit(X_train, y_train)
# y_pred = rfr.predict(X_val)

# r2_score(y_val, y_pred)
rfr.score(X_val, y_val)

0.7350684031039878

In [165]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet
lr = Lasso()

lr.fit(X_train, y_train)
lr.score(X_val, y_val)

0.4727388253356254

In [171]:
df3 = pd.get_dummies(df2.drop(columns=['marca', 'modelo', 'versao']), columns=['tipo', 'cor', 'anunciante', 'estado_vendedor'])

X = df3.drop(columns='preco')
y = df3.preco

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=31)

In [172]:
lr.fit(X_train, y_train)
lr.score(X_val, y_val)

c:\Users\renat\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.425e+12, tolerance: 3.866e+09
  model = cd_fast.enet_coordinate_descent(


0.6448032119818248

In [208]:
from sklearn.svm import SVR

svr_model = SVR()
svr_model.fit(X_train, y_train)
svr_model.score(X_val, y_val)

-0.050788063178260234

In [209]:
coefs = pd.Series(data=lr.coef_, index=lr.feature_names_in_)
# for coef in coefs[coefs.abs() == 0].index:
#     print(coef)
coefs

num_fotos            -124160.443274
ano_de_fabricacao    -121858.921658
ano_modelo           -116227.112250
hodometro            -108406.295993
cambio               -106757.683191
                          ...      
estado_vendedor_20    161908.733668
estado_vendedor_21    164268.510082
estado_vendedor_22    180123.762831
estado_vendedor_23    194536.235591
estado_vendedor_24    659514.572668
Length: 737, dtype: float64

In [211]:
X_train

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,cambio,num_portas,blindado,vendedor_PJ,entrega_delivery,troca,dono_aceita_troca,veiculo_único_dono,revisoes_concessionaria,ipva_pago,veiculo_licenciado,garantia_de_fábrica,revisoes_dentro_agenda,10,100,104,10at,110,120,120cv,122cv,12v,130cv,131cv,140cv,142cv,143cv,150cv,155cv,16,164,16l,16v,170,170cv,180cv,182cv,183cv,184cv,18i,18l,18v,1a11,200,200cv,202cv,20i,20v,21,211cv,24v,25,250,2500,263,270,270cv,285cv,28i,2p,2ss,2wd,300,30d,30s13,30v,311,313,314,32v,330,340cv,35,350,35i,36v,3p,40,40v,415,416,45,4all,4d,4matic,4motion,4p,4s,4sport,4style,4wd,4x2,4x4,4you,50i,515,516,55,5m,60,6a,6i,6v,75,7g,7l,800kg,8v,90,94cv,97cv,9g,absolute,act,action,activ,activ7,active,activeflex,advance,advantage,adventure,air,all4,allgrip,allspace,allure,altis,alto,ambiente,ambition,amg,anos,at6,at8,at9,atacama,attack,attitude,attraction,attractive,audace,authentique,autobiography,automatico,automatizado,automático,automátizado,avant,avantgarde,awc,awd,b12p,base,be,bev,bi,biturbo,black,blackjack,bluehdi,bluemotion,bluetec,boost,bose,boxster,brabus,business,bva,cab,cabine,cabrio,cabriolet,cargo,cargo16v,carrera,cat,cayman,caçamba,cc,cd,cdi,ce,cgi,challenge,chassi,chassis,chic,chilli,cilindros,city,class,classic,clubman,colina,com,combinato,comfort,comfortline,compact,competition,confortline,connect,conquest,conversível,cooper,copa,country,coupe,coupé,coupê,cross,crosstrek,crossway,crx,cs,cult,curl,curto,custom,cvt,d180,d240,d300,d5,dark,dci,dct,de,del,design,diamond,diesel,direct,dlx,do,drive,dsg,dx,dynamic,dynamique,eagle,easy,eat6,ecoboost,econo,economy,ecoshift,ecotec,edition,edição,edrive,efi,el,electric,electronic,elegance,elegant,eletronic,elevado,elite,elx,elétrico,endurance,enduro,engineered,eq,escapade,especial,essence,estate,ev,ev400,evo,evolution,ex,ex2,ex3,excellence,exclusive,executive,exl,express,expression,exr,exs,extra,extreme,f1,fastback,feel,feline,fifa,fifty,fire,firefly,first,flex,flexone,flexstart,forest,freedom,freeride,freestyle,fsi,full,furgão,fwd,gas,gasolina,gdi,gear,geartronic,ghia,gl,gli,gls,glx,gp,gr,gran,grand,graphite,griffe,gsr,gsx,gt,gtdi,gte,gti,gts,guard,gx,h4,h6,hard,hatch,hd,hdi,hemi,hgt,hi,high,highline,highway,hl,hle,hls,hpe,hpi,hr,hse,hybrid,híbrido,i4,i6,iconic,ie,iflex,ii,iii,impetus,impress,in,ingenium,inscription,intense,intensive,intercoler,intercooler,iv,jet,jetflex,john,joy,k788,kinetic,kompressor,koup,kw,l1h1,l2h1,l2h2,l3h2,label,landmark,laramie,laredo,last,launch,launching,le,life,like,limitada,limited,line,lineartronic,live,locker,longitude,longo,look,lotação,ls,lt,ltz,lugares,luxe,luxury,lx,lxl,lxr,lxs,m40i,m50d,m50i,mach,manual,maxicargo,maxx,mct,mhev,mi,midnight,mille,million,minibus,mivec,moab,momentum,motion,move,mpfi,mpi,msi,multiair,multidrive,multijet,multitronic,mundo,médio,night,ocean,open,origine,outdoor,outsider,overland,p250,p300,p380,p550,p8,pack,pallas,passion,pdk,pepper,performance,personal,phev,picasso,pick,platinum,plus,polestar,portfolio,power,powershift,precision,premier,premium,presence,prestige,prime,pro,progressive,pulse,pure,q4,quattro,racing,ralliart,rallye,ranch,rbw,rebel,recharge,rex,rigido,rio,rip,road,roadster,robust,rocam,rock,rs,rt,rts,run,rx,rxs,rxt,rígido,sahara,sce,sd4,sda,sdrive18i,sdrive20i,sdrive30i,sdv8,se,security,sedan,sel,selectshift,sem,sense,series,sfi,shadow,shift,shine,si,si4,sidi,sierra,sigma,sl,slt,smart,so,sol,spe4,spec,special,speed,speedshift,spider,spirit,sport,sport6,sportback,sporting,sportline,sportrac,sr,srv,srx,ss,start,std,steptronic,stepway,sti,storm,street,stx,style,super,supercharged,surf,sv,svautobiography,svr,sw,sx,sx3,sxt,série,t270,t3,t4,t5,t6,t8,ta,take,targa,tci,td,td350,td380,td4,td6,tdi,tdv6,tdv8,tech,tecno,tendance,teto,tetrafuel,tfsi,tgdi,tgv,thp,ti,tiptronic,titanium,top,tornado,torq,total,totalflex,touring,track,trailhawk,trekking,trend,trendline,tronic,trooper,tropical,tropivan,tsi,turbo,turbocharged,turismo,tweed,twin,twinpower,twinturbo,txs,ue71,ultimate,ultra,unique,unlimited